# Лабораторная работа 5
# Ансамбли моделей машинного обучения.
# Задание
1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите следующие ансамблевые модели:
  - две модели группы бэггинга (бэггинг или случайный лес или сверхслучайные деревья);
   -AdaBoost;
   -градиентный бустинг.
5. Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей.


# Импорт бибилиотек

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# Импорт данных

In [10]:
data = pd.read_csv('winequality-red.csv', encoding = 'ISO-8859-1')

In [11]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


# Обработка данных

Типы данных в представленном датасете:

In [12]:
data.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

Проверим, есть ли в датасете пропущенные значения:

In [13]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

В данном датасете нет строк или столбцов, содержащих пропущенные значения.

In [14]:
data.columns.tolist()

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [15]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Проверим, содержатся ли в данных дубликаты:

In [16]:
print(data.duplicated())

0       False
1       False
2       False
3       False
4        True
        ...  
1594    False
1595    False
1596     True
1597    False
1598    False
Length: 1599, dtype: bool


In [17]:
duplicate_rows = data[data.duplicated()]
print(duplicate_rows)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
4               7.4             0.700         0.00            1.90      0.076   
11              7.5             0.500         0.36            6.10      0.071   
27              7.9             0.430         0.21            1.60      0.106   
40              7.3             0.450         0.36            5.90      0.074   
65              7.2             0.725         0.05            4.65      0.086   
...             ...               ...          ...             ...        ...   
1563            7.2             0.695         0.13            2.00      0.076   
1564            7.2             0.695         0.13            2.00      0.076   
1567            7.2             0.695         0.13            2.00      0.076   
1581            6.2             0.560         0.09            1.70      0.053   
1596            6.3             0.510         0.13            2.30      0.076   

      free sulfur dioxide  

В данном датасете содержатся дубликаты. Удалим эти строки:

In [18]:
data = data.drop_duplicates()

In [19]:
print(data.duplicated())
duplicate_rows = data[data.duplicated()]
print(duplicate_rows)

0       False
1       False
2       False
3       False
5       False
        ...  
1593    False
1594    False
1595    False
1597    False
1598    False
Length: 1359, dtype: bool
Empty DataFrame
Columns: [fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH, sulphates, alcohol, quality]
Index: []


Теперь дубликатов нет.

In [20]:
data['quality'].unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [21]:
le = LabelEncoder()
data['quality'] = le.fit_transform(data['quality'])

In [22]:
data['quality'].unique()

array([2, 3, 4, 1, 5, 0], dtype=int64)

# Разделение выборки на обучающую и тестовую

С использованием метода train_test_split разделим выборку на обучающую и тестовую:

In [23]:
data.shape

(1359, 12)

In [24]:
# Признаки без целевой переменной
x = data.drop('quality', axis=1)
x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4


In [25]:
# Целевая переменная
y = data['quality']
y.head()

0    2
1    2
2    2
3    3
5    2
Name: quality, dtype: int64

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
#20% данных будут отложены для тестирования, а 80% будут использованы для обучения модели

In [27]:
# Размер обучающей выборки
x_train.shape, y_train.shape

((1087, 11), (1087,))

In [28]:
# Размер тестовой выборки
x_test.shape, y_test.shape

((272, 11), (272,))

# Обучение ансамблевых моделей

## Бэггинг

### Бэггинг

In [29]:
model_bc = BaggingClassifier(n_estimators=200, oob_score=True, random_state=10)
model_bc.fit(x_train, y_train)

BaggingClassifier(n_estimators=200, oob_score=True, random_state=10)

### Случайный лес

In [30]:
rand_forest_model = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=10)
rand_forest_model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=10)

## AdaBoost

In [31]:
ada_boost_model = AdaBoostClassifier(n_estimators=200, algorithm='SAMME', random_state=10)
ada_boost_model.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME', n_estimators=200, random_state=10)

## Градиентный бустинг


In [32]:
grad_boost_model = GradientBoostingClassifier(n_estimators=200)
grad_boost_model.fit(x_train, y_train)

GradientBoostingClassifier(n_estimators=200)

## Стекинг

In [33]:
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=10)),
     ('svc', make_pipeline(StandardScaler(),
                           SVC(random_state=10)))]
stk_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stk_model.fit(x_train, y_train)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=10)),
                               ('svc',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('svc',
                                                 SVC(random_state=10))]))],
                   final_estimator=LogisticRegression())

# Многослойный персептрон

In [34]:
from sklearn.neural_network import MLPClassifier

In [35]:
model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100)
model.fit(x_train, y_train)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100)

# Оценка качества моделей

In [36]:
models = [model_bc, rand_forest_model, ada_boost_model, grad_boost_model, stk_model, model]
for model in models:
    print(f"{type(model).__name__}:")
    print('\t',f"accuracy = {accuracy_score(y_test,model.predict(x_test))}")

BaggingClassifier:
	 accuracy = 0.5808823529411765
RandomForestClassifier:
	 accuracy = 0.5698529411764706
AdaBoostClassifier:
	 accuracy = 0.47058823529411764
GradientBoostingClassifier:
	 accuracy = 0.5551470588235294
StackingClassifier:
	 accuracy = 0.5661764705882353
MLPClassifier:
	 accuracy = 0.5551470588235294
